In [7]:
from pysmt.shortcuts import *
import pysmt.typing as type
import random
from pysmt.typing import BOOL, REAL, INT, BVType, STRING

n = 32  # número de bits das variáveis

# Função para declarar as variáveis de estado
def declare(i):
    state = {}    
    state['pc'] = Symbol('pc'+str(i),BVType(n))
    state['r'] = Symbol('r'+str(i),BVType(n))
    state['s'] = Symbol('s'+str(i),BVType(n))
    state['t'] = Symbol('t'+str(i),BVType(n))
    state['q'] = Symbol('q'+str(i),BVType(n))
    state['r_'] = Symbol('r_'+str(i),BVType(n))
    state['s_'] = Symbol('s_'+str(i),BVType(n))
    state['t_'] = Symbol('t_'+str(i),BVType(n))
    return state

# Função de inicialização
def init(state, a, b):
    A = BVUGT(state['r'], SBV(0, n))
    B = BVUGT(state['r_'], SBV(0, n))
    C = Equals(state['pc'], SBV(0, n))
    D = Equals(state['r'], SBV(a, n))
    E = Equals(state['s'], SBV(1, n))
    F = Equals(state['t'], SBV(0, n)) 
    G = Equals(state['r_'], SBV(b, n))
    H = Equals(state['s_'], SBV(0, n))
    I = Equals(state['t_'], SBV(1, n))
    J = Equals(state['q'], SBV(0, n))
    
    r = And(A, B, C, D, E, F, G, H, I, J)
    
    return r

# Função de transição entre estados
def trans(curr, prox):
    # 0 -> 1
    A = Equals(curr['pc'], SBV(0, n))
    B = Equals(prox['pc'], SBV(1, n))
    C = Equals(prox['r'], curr['r'])
    D = Equals(prox['s'], curr['s'])
    E = Equals(prox['t'], curr['t'])
    F = Equals(prox['r_'], curr['r_'])
    G = Equals(prox['s_'], curr['s_'])
    H = Equals(prox['t_'], curr['t_'])
    I = Equals(prox['q'], curr['q'])
    t01 = And(A, B, C, D, E, F, G, H, I)
    
    # 1 -> 2
    A = Equals(curr['pc'], SBV(1, n))
    B = Equals(prox['pc'], SBV(2, n))
    C = Equals(prox['r'], curr['r'])
    D = Equals(prox['s'], curr['s'])
    E = Equals(prox['t'], curr['t'])
    F = Equals(prox['r_'], curr['r_'])
    G = Equals(prox['s_'], curr['s_'])
    H = Equals(prox['t_'], curr['t_'])
    z = Not(Equals(curr['r_'], SBV(0, n)))
    t12 = And(A, B, C, D, E, F, G, H, z)
    
    # 2 -> 1
    A = Equals(curr['pc'], SBV(2, n))
    B = Equals(prox['pc'], SBV(1, n))
    D = Equals(prox['q'], BVSDiv(curr['r'], curr['r_']))
    E = Equals(prox['r_'], BVSub(curr['r'], BVMul(prox['q'], curr['r_'])))
    F = Equals(prox['s_'], BVSub(curr['s'], BVMul(prox['q'], curr['s_'])))
    G = Equals(prox['t_'], BVSub(curr['t'], BVMul(prox['q'], curr['t_'])))
    H = Equals(prox['r'], curr['r_'])
    I = Equals(prox['s'], curr['s_'])
    J = Equals(prox['t'], curr['t_'])
    t21 = And(A, B, D, E, F, G, H, I, J)
    
    # 1 -> 3
    A = Equals(curr['pc'], SBV(1, n))
    B = Equals(prox['pc'], SBV(3, n))
    C = Equals(curr['r_'], SBV(0, n))
    D = Equals(prox['r'], curr['r'])
    E = Equals(prox['s'], curr['s'])
    F = Equals(prox['t'], curr['t'])
    G = Equals(prox['r_'], curr['r_'])
    H = Equals(prox['s_'], curr['s_'])
    I = Equals(prox['t_'], curr['t_'])
    J = Equals(prox['q'], curr['q'])
    t13 = And(A, B, C, D, E, F, G, H, I, J)
    
    # 3 -> 3
    A = Equals(curr['pc'], SBV(3, n))
    B = Equals(prox['pc'], SBV(3, n))
    C = Equals(prox['r'], curr['r'])
    D = Equals(prox['s'], curr['s'])
    E = Equals(prox['t'], curr['t'])
    F = Equals(prox['r_'], curr['r_'])
    G = Equals(prox['s_'], curr['s_'])
    H = Equals(prox['t_'], curr['t_'])
    I = Equals(prox['q'], curr['q'])
    t33 = And(A, B, C, D, E, F, G, H, I)
    
    return Or(t01, t12, t21, t13, t33)






# Função para gerar o traço
def gera_traco(declare, init, trans, k, a, b):
    if a > 0 and b > 0:
        with Solver() as solver:
            traco = [declare(i) for i in range(k)]

            solver.add_assertion(init(traco[0], a, b))

            for i in range(k - 1):
                solver.add_assertion(trans(traco[i], traco[i + 1]))

            if solver.solve():
                print("> is sat")
                for i, s in enumerate(traco):
                    print("Estado pc: %s" % (solver.get_value(s['pc']).bv_signed_value()))
                    print("Valor R: %s" % (solver.get_value(s['r']).bv_signed_value()))
                    print("Valor R_: %s" % (solver.get_value(s['r_']).bv_signed_value()))
                    print("Valor S: %s" % (solver.get_value(s['s']).bv_signed_value()))
                    print("Valor S_: %s" % (solver.get_value(s['s_']).bv_signed_value()))
                    print("Valor T: %s" % (solver.get_value(s['t']).bv_signed_value()))
                    print("Valor T_: %s" % (solver.get_value(s['t_']).bv_signed_value()))
                    print("Valor Q: %s" % (solver.get_value(s['q']).bv_signed_value()))   
                    print("")
            else:
                print("> Not feasible.")

# Função para verificar se r != 0
def r_dif_zero(state):
    return Not(Equals(state['r'], SBV(0, n)))

# Função para verificar se não há overflow
def no_overflow(state):     
    N = BV(2**n-1, width=n)
    A = Or(
        BVUGT(state['r'], N),
        BVUGT(state['s'], N),
        BVUGT(state['t'], N),
        BVUGT(state['r_'], N),
        BVUGT(state['s_'], N),
        BVUGT(state['t_'], N),
        BVUGT(state['q'], N)
    )
    return Not(A)

# Função de invariante de não erro
def nao_erro(state):
    return And(r_dif_zero(state), no_overflow(state))

def interpolante_prova(solver, X, Y, a, b, order, error):
    for (n, m) in order:
        I = init(X[0], a, b)
        Tn = And([trans(X[i], X[i+1]) for i in range(n)])
        Rn = And(I, Tn)
        
        E = error(Y[0])
        Bm = And([trans(Y[i], Y[i+1]) for i in range(m)])
        Um = And(E, Bm)
        
        # Calculando o interpolante entre Rn e Um
        C = binary_interpolant(Rn, Um)
        
        # **Print the interpolant**
        print("> Interpolant C:", C)
        
        # Se o interpolante for None, a prova de segurança falha.
        if C is None:
            print("> O interpolante é None.")
            break
        
        # Verificando se o interpolante é invariante
        C0 = rename(C, X[0])
        T = trans(X[0], X[1])
        C1 = rename(C, X[1])

        if not solver.solve([C0, T, Not(C1)]):
            # Se não encontrar contraexemplo, o sistema é seguro
            print("> O sistema é seguro.")
            return 
        else:
            # Caso contrário, tentamos encontrar um maiorante
            S = rename(C, X[n])
            while True:
                T = trans(X[n], Y[m])
                A = And(S, T)
                if solver.solve([A, Um]):
                    print("> Não foi encontrado majorante.")
                    break 
                else:
                    # Calculamos novamente o interpolante
                    C = binary_interpolant(A, Um)
                    # **Print the new interpolant**
                    print("> New Interpolant C:", C)
                    Cn = rename(C, X[n])
                    if not solver.solve([Cn, Not(S)]):
                        print("> O sistema é seguro.")
                        return
                    else:
                        # Caso contrário, aumentamos o conjunto de estados
                        S = Or(S, Cn)

def verifica_sistema_com_interpolacao(declare, init, trans, error, a, b, order, k, solver):
    X = [declare(i) for i in range(k)]
    Y = [declare(i + k) for i in range(k)]
    interpolante_prova(solver, X, Y, a, b, order, error)
    
    

# Função para verificação por k-indução
def k_induction_always(declare, init, trans, inv, k, a, b):
    if a > 0 and b > 0:
        with Solver() as solver:
            s = [declare(i) for i in range(k)]
            solver.add_assertion(init(s[0], a, b))
            for i in range(k - 1):
                solver.add_assertion(trans(s[i], s[i + 1]))

            for i in range(k):
                solver.push()
                solver.add_assertion(Not(inv(s[i])))
                if solver.solve():
                    print(f"> Contradição! O invariante não se verifica nos k estados iniciais.")
                    for st in s:
                        print(f" pc = {solver.get_value(st['pc']).bv_signed_value()}", end=" ")
                        print(f" r = {solver.get_value(st['r']).bv_signed_value()}", end=" ")
                        print(f" s = {solver.get_value(st['s']).bv_signed_value()}", end=" ")
                        print(f" t = {solver.get_value(st['t']).bv_signed_value()}")
                        print(f" r_ = {solver.get_value(st['r_']).bv_signed_value()}", end=" ")
                        print(f" s_ = {solver.get_value(st['s_']).bv_signed_value()}", end=" ")
                        print(f" t_ = {solver.get_value(st['t_']).bv_signed_value()}", end=" ")
                        print(f" q = {solver.get_value(st['q']).bv_signed_value()}", end=" ")
                        print("---------------------------------------------------------")
                        print()

                    return
                solver.pop()

            s2 = [declare(i + k) for i in range(k + 1)]

            for i in range(k):
                solver.add_assertion(inv(s2[i]))
                solver.add_assertion(trans(s2[i], s2[i + 1]))

            solver.add_assertion(Not(inv(s2[-1])))

            if solver.solve():
                print(f"> Contradição! O passo indutivo não se verifica.")
                return
            print(f"> A propriedade verifica-se por k-indução (k={k}).")

# Gerar traço de execução
gera_traco(declare, init, trans, 7, 3, 6)
with Solver() as solver:
    order = [(2, 2)]  # Adjust (n, m) pairs as needed
    verifica_sistema_com_interpolacao(declare, init, trans, nao_erro, 3, 6, order, 7, solver)
# Verificar a propriedade por k-indução
k_induction_always(declare, init, trans, nao_erro, 7, 1, 5)


> is sat
Estado pc: 0
Valor R: 3
Valor R_: 6
Valor S: 1
Valor S_: 0
Valor T: 0
Valor T_: 1
Valor Q: 0

Estado pc: 1
Valor R: 3
Valor R_: 6
Valor S: 1
Valor S_: 0
Valor T: 0
Valor T_: 1
Valor Q: 0

Estado pc: 2
Valor R: 3
Valor R_: 6
Valor S: 1
Valor S_: 0
Valor T: 0
Valor T_: 1
Valor Q: 0

Estado pc: 1
Valor R: 6
Valor R_: 3
Valor S: 0
Valor S_: 1
Valor T: 1
Valor T_: 0
Valor Q: 0

Estado pc: 2
Valor R: 6
Valor R_: 3
Valor S: 0
Valor S_: 1
Valor T: 1
Valor T_: 0
Valor Q: 0

Estado pc: 1
Valor R: 3
Valor R_: 0
Valor S: 1
Valor S_: -2
Valor T: 0
Valor T_: 1
Valor Q: 2

Estado pc: 3
Valor R: 3
Valor R_: 0
Valor S: 1
Valor S_: -2
Valor T: 0
Valor T_: 1
Valor Q: 2

> Interpolant C: ((((0_32 u< r0) & (0_32 u< r_0) & (pc0 = 0_32) & (r0 = 3_32) & (s0 = 1_32) & (t0 = 0_32) & (r_0 = 6_32) & (s_0 = 0_32) & (t_0 = 1_32) & (q0 = 0_32)) & (((... & ... & ... & ... & ... & ... & ... & ... & ...) | (... & ... & ... & ... & ... & ... & ... & ... & ...) | (... & ... & ... & ... & ... & ... & ... & ... & 